# Plot crashes in Hudson County, 2017-2021 

In [ ]:
from utz import *
from njdot.crashes import Crashes
import plotly.express as px
import plotly.graph_objects as go

## Mapbox configs

In [ ]:
token = open(".mapbox-token").read()
mapbox = dict(mapbox=dict(
    style="dark",
    accesstoken=token,
))

In [ ]:
%%time
first_year, last_year = 2017, 2021
years = list(range(first_year, last_year + 1))

## Load all counties, filter to Hudson by geometry
(some crashes' "County Name"s seem miscategorized, based on lat/lng)

In [ ]:
%%time
c = Crashes.load(years=years)
c

In [ ]:
c.severity.value_counts()

### Compute LLs

In [ ]:
%%time
cl = c.lls()
cl

In [ ]:
cl.isna().sum()

### Load county geometries

In [ ]:
from nj_crashes.geo import get_boundary_lls, get_county_coords, get_counties
from geopandas.tools import sjoin
import geopandas as gpd
from shapely import box
import simplejson
from njdot.codes import CrashSeverity

In [ ]:
counties = get_counties()
counties

### Hudson County geometry

In [ ]:
hudco = counties.loc[counties.NAMELSAD == 'Hudson County']
hudco

In [ ]:
counties.to_file(f'www/public/counties.geojson', driver='GeoJSON')

In [ ]:
hudco.to_file(f'www/public/hudson.geojson', driver='GeoJSON')

In [ ]:
hudco.geometry.iloc[0]

In [ ]:
gdf = gpd.GeoDataFrame(
    cl,
    geometry=gpd.points_from_xy(x=cl.lon, y=cl.lat),
    columns=cl.columns,
).astype(cl.dtypes)
gdf

In [ ]:
type(hudco), type(hudco[['geometry']])

### Hudson County crashes (by geometry)

In [ ]:
hp = sjoin(gdf, hudco[['geometry']])
hp.severity = hp.severity.apply(lambda s: s[0].lower())
hp

In [ ]:
hp_hist = hp.severity.value_counts()
hp_hist

In [ ]:
hc = c[c['County Name'] == 'HUDSON']
len(hc)

In [ ]:
hcs_hist = hc.severity.value_counts()
hcs_hist

In [ ]:
hcs_ksi = hcs_hist.Injury + hcs_hist.Fatal
hp_ksi = hp_hist.i + hp_hist.f
hcs_ksi, hp_ksi, hcs_ksi - hp_ksi

In [ ]:
hcs_hist.Injury + hcs_hist.Fatal - 

In [ ]:
hc[[ 'Total Killed', 'Total Injured', 'Total Vehicles Involved', ]].sum()

Peek at injury + fatal crashes:

In [ ]:
hp_if = hp[(hp.severity == 'i') | (hp.severity == 'f')]
hp_if

In [ ]:
diff = hp_if[(hp_if.lat != hp_if.oilat) | (hp_if.oilon != hp_if.lon)]
diff

## Export crashes

### Reduce + export JSON

In [ ]:
from nj_crashes.json import reduce_df

In [ ]:
outdir = 'www/public/plots/njdot'

def reduce_and_save(df, name=None, renames=None, cols=None):
    path = f'{outdir}/{name}' if name else None
    if renames:
        df = df.rename(columns=renames)
    return reduce_df(df, cols=cols, path=path)

In [ ]:
renames = {
    'Date': 'dt',
    'Municipality Name': 'city',
    'Total Killed': 'tk',
    'Total Injured': 'ti',
    'Pedestrians Killed': 'pk',
    'Pedestrians Injured': 'pi',
    'Total Vehicles Involved': 'tv',
}
cols = {
    'dt': dict(dt_unit='m'),
    'city': True,
    'tk': False,
    'ti': False,
    'pk': False,
    'pi': False,
    'severity': True,
    'tv': False,
    'sri': True,
    'mp': False,
    'lon': True,
    'lat': True,
}
fi, obj = reduce_and_save(
    hp_if,
    name=f'hudson-5yr-lls-if.json',
    renames=renames,
    cols=cols,
)

Reduced rows:

In [ ]:
obj['rows'][:10]

Cols:

In [ ]:
obj['cols']

Dictionaries:

In [ ]:
obj['dicts']

In [ ]:
rd, do = reduce_and_save(
    diff,
    name=f'hudson-5yr-lls-if-diffs.json',
    cols={ **cols, 'oilat': True, 'oilon': True },
    renames=renames,
)
rd

## Check County Names

Check for crashes labeled as happening outside Hudson County:

In [ ]:
hp['County Name'].value_counts()

In [ ]:
hp['Municipality Name'].value_counts()

## Plot crashes

### Compute bounding box, center

In [ ]:
bb = hudco.bounds.values[0]
min_lon, min_lat, max_lon, max_lat = bb
bb = box(*bb)
centroid = bb.centroid
mid_lon, mid_lat = centroid.x, centroid.y

min_lon, min_lat, max_lon, max_lat

### Compute lat/lng cluster sizes

In [ ]:
from njdot.crashes import LLCrashes

In [ ]:
ll_hist = LLCrashes(hp).ll_hist
ll_hist

In [ ]:
ll_hist[['severity', 'lls_count']].value_counts(sort=False)

### Plot helpers

In [ ]:
from IPython.display import display, Image

In [ ]:
severities = list(CrashSeverity.ch2Name.values())

In [ ]:
year_str = f"{first_year}" if first_year == last_year else f"{first_year}-{last_year}"
year_str

Plot crashes, with "density" / shadow, and optional exported/inline display formats

In [ ]:
def crash_plot(
    lls,
    county,
    types='pif',
    center=None, zoom=11.4,
    lg='tr', lm=0.02,
    colors=None, scale_radii=None, size='radius',
    marker=None,
    legend=None,
    W=1000, H=1000,
    layers=None,
    cluster=None,
    density=None,
    density_radius=20,
    zmaxs=None,
    write=True,
    map='carto-darkmatter',  # 'dark' will load .mapbox-token
    show=False,
    print_stats=False,
    **kwargs
):
    name = f'{outdir}/{county.lower()}-{types}-crashes'
    scale_radii = scale_radii or [1, 2, 4]
    colors = colors or ['yellow', 'orange', 'red']
    zmaxs = zmaxs or [20, 10, 1]
    center = center or dict(lon=mid_lon, lat=mid_lat)
    if isinstance(center, list):
        center = dict(lon=center[0], lat=center[1])
    severity_dfs = []
    density_traces = []
    for idx, ((ch, severity), color, scale_radius, zmax) in enumerate(zip(CrashSeverity.ch2Name.items(), colors, scale_radii, zmaxs)):
        if ch not in types:
            continue
        severity_df = lls[lls.severity == severity]
        severity_df = severity_df.assign(radius=severity_df.radius * scale_radius)
        severity_dfs.append(severity_df)

        if density:
            density_trace = go.Densitymapbox(
                name=f'{severity} density',
                lon=severity_df.lon, lat=severity_df.lat,
                zmin=0, zmax=zmax,
                radius=density_radius,
                colorscale=[[0, 'black'], [1, color]],
                hoverinfo='none',
                showscale=False,
                legendrank=990+idx,
            )
            density_traces.append(density_trace)
    df = pd.concat(severity_dfs)
    if print_stats:
        num_crashes_by_type = df.severity.value_counts()
        num_crashes = num_crashes_by_type.sum()
        print(f"{num_crashes} crashes:")
        print(str(num_crashes_by_type))
    fig = px.scatter_mapbox(
        df,
        lon='lon', lat='lat',
        size=size,
        #size_max=10,
        color='severity',
        color_discrete_map=dict(zip(severities, colors)),
        hover_data=['Date', 'Crash Location', 'sri', 'mp', 'lls_count',],
        center=center,
        zoom=zoom,
        height=H,
        **kwargs
    )
    legend_bgcolor = '50'
    mapbox_kwargs = dict(style=map)
    if map == 'dark':
        token = open(".mapbox-token").read()
        mapbox_kwargs['accesstoken'] = token
    if density_traces:
        for trace in density_traces:
            fig.add_trace(trace)
        fig.data = fig.data[-len(density_traces):] + fig.data[:len(density_traces)]
    if lg == 'tr':
        legend_pos = dict(yanchor="top", xanchor="right", x=1-lm, y=1-lm)
    elif lg == 'br':
        legend_pos = dict(yanchor="bottom", xanchor="right", x=1-lm, y=lm)
    else:
        raise ValueError(f'Unrecognized legend position: {lg}')
    legend_obj = dict(
        title=dict(text=''),
        **legend_pos,
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    )
    if legend:
        legend_obj.update(legend)
    legend = legend_obj
    label = ' KSI' if types == 'if' else ' fatal' if types == 'f' else ''
    fig.update_layout(
        mapbox=dict(
            # dict(style=<style>) or dict(style="dark", accesstoken=token)
            **mapbox_kwargs,
            layers=layers,
            #layers=[ { "sourcetype": "image", "source": img, "coordinates": coordinates }, ]
        ),
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        title=dict(
            text=f"{county} County{label} crashes ({year_str})",
            x=0.5, y=0.98,
            xanchor='center', yanchor='top',
            font=dict(size=32, color="white")
        ),
        legend=legend,
    )
    scatter_kwargs = dict(
        cluster=cluster,
        #marker=marker if marker else None,
    )
    for idx, (severity, color) in enumerate(zip(severities, colors)):
        fig.update_traces(selector=dict(name=severity), legendrank=1001 + idx, marker=dict(color=color), **scatter_kwargs)
    if cluster:
        fig.update_traces(cluster=dict(enabled=True))
    if write is True or write == 'png':
        path = f'{name}.png'
        fig.write_image(path, width=W, height=H)
        err(f"Wrote {path}")
    if write is True or write == 'json':
        path = f'{name}.json'
        fig.write_json(path)
        err(f"Wrote {path}")
    if show == 'img':
        return Image(fig.to_image(width=W, height=H))
    elif show:
        fig.show(config=dict(displayModeBar=False))
    return fig

In [ ]:
args = (ll_hist.df, 'Hudson',)
show = 'img'
# show = False
def plot_mix(types='pif', **kwargs):
    kw = dict(
        size_max=15,
        scale_radii=[2, 3, 6],
        types=types,
        density=True,
        zoom=11.5,
        lg='br',
        H=1100,
        show=show,
    )
    kw.update(**kwargs)
    crash_plot(*args, **kwargs, map='dark', write='png', print_stats=True, **kw)
    return crash_plot(*args, **kwargs, write='json', **kw)

### Plot crashes by type

In [ ]:
plot_mix('pif')

In [ ]:
plot_mix('if')

In [ ]:
plot_mix('f')

Simple injury+fatal plot, without "density":

In [ ]:
crash_plot(ll_hist, 'Hudson', types='if', size='radius', size_max=10, show='img')